In [1]:
pip install psycopg2

In [2]:
# import libraries
import pandas as pd
import os
import sqlite3
from sqlalchemy import create_engine

In [3]:
db_config = {'user': 'praktikum_student',         # user name
             'pwd': 'Sdf4$2;d-d30pp', # password
             'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
             'port': 6432,              # connection port
             'db': 'data-analyst-final-project-db'}          # the name of the data base

connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
                                                                     db_config['pwd'],
                                                                       db_config['host'],
                                                                       db_config['port'],
                                                                       db_config['db'])

In [4]:
engine = create_engine(connection_string, connect_args={'sslmode':'require'})

In [5]:
cnn=sqlite3.connect('data-analyst-final-project-db')

In [6]:
books_df=pd.read_sql_query("SELECT * FROM books", con=connection_string)

In [7]:
books_df

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268
...,...,...,...,...,...,...
995,996,571,Wyrd Sisters (Discworld #6; Witches #2),265,2001-02-06,147
996,997,454,Xenocide (Ender's Saga #3),592,1996-07-15,297
997,998,201,Year of Wonders,358,2002-04-30,212
998,999,94,You Suck (A Love Story #2),328,2007-01-16,331


In [8]:
auth_df=pd.read_sql_query("SELECT * FROM authors", con=connection_string)
auth_df

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd
...,...,...
631,632,William Strunk Jr./E.B. White
632,633,Zadie Smith
633,634,Zilpha Keatley Snyder
634,635,Zora Neale Hurston


In [9]:
publi_df=pd.read_sql_query("SELECT * FROM publishers", con=connection_string)
publi_df

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company
...,...,...
335,336,Workman Publishing Company
336,337,Wyatt Book
337,338,Yale University Press
338,339,Yearling


In [10]:
rating_df=pd.read_sql_query("SELECT * FROM ratings", con=connection_string)
rating_df

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2
...,...,...,...,...
6451,6452,1000,carolrodriguez,4
6452,6453,1000,wendy18,4
6453,6454,1000,jarvispaul,5
6454,6455,1000,zross,2


In [11]:
review_df=pd.read_sql_query("SELECT * FROM reviews", con=connection_string)
review_df

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...
...,...,...,...,...
2788,2789,999,martinadam,Later hospital turn easy community. Fact same ...
2789,2790,1000,wknight,Change lose answer close pressure. Spend so now.
2790,2791,1000,carolrodriguez,Authority go who television entire hair guy po...
2791,2792,1000,wendy18,Or western offer wonder ask. More hear phone f...


##### Find the number of books released after January 1, 2000.

In [12]:
df=pd.read_sql_query(("""SELECT count(*) 
                     from books where publication_date > '01-01-2000'"""), con=connection_string)
df

,count
0,819


###### There are 819 books released after January 1, 2000.

##### Find the number of user reviews and the average rating for each book.

In [13]:
df=pd.read_sql_query(("""SELECT books.title, COUNT(review_id) as num_reviews 
                     FROM books 
                     LEFT JOIN reviews ON books.book_id=reviews.book_id 
                     GROUP BY books.title"""),
                     con=connection_string)
df

,title,num_reviews
0,The Count of Monte Cristo,5
1,Count Zero (Sprawl #2),2
2,The Botany of Desire: A Plant's-Eye View of th...,2
3,The Poisonwood Bible,5
4,The Canterbury Tales,3
...,...,...
994,Of Love and Other Demons,2
995,In the Heart of the Sea: The Tragedy of the Wh...,3
996,Welcome to Temptation (Dempseys #1),2
997,World's End (The Sandman #8),2


In [14]:
df=pd.read_sql_query(("""SELECT b.title, reviews.num_reviews, ratings.avg_rating
                         FROM books AS b
                         LEFT JOIN (SELECT b.title, COUNT(review_id) as num_reviews 
                                    FROM books AS b
                                    LEFT JOIN reviews ON b.book_id=reviews.book_id 
                                    GROUP BY b.title)
                         AS reviews ON b.title = reviews.title
                         LEFT JOIN(SELECT book_id, AVG(rating) as avg_rating
                                   FROM ratings
                                   GROUP BY book_id)
                         ratings ON b.book_id = ratings.book_id
                         ORDER BY reviews.num_reviews DESC"""), con=connection_string)
df

,title,num_reviews,avg_rating
0,Memoirs of a Geisha,8,3.700000
1,Memoirs of a Geisha,8,4.195652
2,Twilight (Twilight #1),7,3.662500
3,Harry Potter and the Prisoner of Azkaban (Harr...,6,4.414634
4,The Lightning Thief (Percy Jackson and the Oly...,6,4.080645
...,...,...,...
995,The Cat in the Hat and Other Dr. Seuss Favorites,0,5.000000
996,The Natural Way to Draw,0,3.000000
997,Essential Tales and Poems,0,4.000000
998,Leonardo's Notebooks,0,4.000000


###### The books with the highest number of reviews do not always have a high average rating. We also notice that the title of the book is repeated.

##### Identify the publisher that has released the greatest number of books with more than 50 pages (this will help you exclude brochures and similar publications from your analysis).

In [15]:
df=pd.read_sql_query(("""SELECT publisher, COUNT(books.book_id) AS num_books
                        FROM publishers AS p
                        INNER JOIN books ON p.publisher_id = books.publisher_id
                        WHERE books.num_pages > 50
                        GROUP BY p.publisher_id
                        ORDER BY num_books DESC"""), con=connection_string)
df

,publisher,num_books
0,Penguin Books,42
1,Vintage,31
2,Grand Central Publishing,25
3,Penguin Classics,24
4,Ballantine Books,19
...,...,...
329,Ballantine Books (NY),1
330,Plaza y Janés,1
331,HarperCollinsPublishers,1
332,Random House: Modern Library,1


###### Penguin Books has published 42 books, which has the greatest number of books with more than 50 pages.

##### Identify the author with the highest average book rating (look only at books with at least 50 ratings).

In [16]:
df=pd.read_sql_query(("""SELECT a.author_id, a.author, AVG(ratings.rating) AS avg_rating 
                          FROM authors AS a
                          JOIN books ON a.author_id=books.author_id
                          JOIN ratings ON books.book_id=ratings.book_id
                          GROUP BY a.author_id, a.author
                          HAVING COUNT(ratings.rating_id) >= 50
                          ORDER BY avg_rating DESC
                          """), con=connection_string)
df

,author_id,author,avg_rating
0,130,Diana Gabaldon,4.300000
1,236,J.K. Rowling/Mary GrandPré,4.288462
2,3,Agatha Christie,4.283019
3,402,Markus Zusak/Cao Xuân Việt Khương,4.264151
4,240,J.R.R. Tolkien,4.240964
5,499,Roald Dahl/Quentin Blake,4.209677
6,376,Louisa May Alcott,4.203704
7,498,Rick Riordan,4.130952
8,39,Arthur Golden,4.107143
9,542,Stephen King,4.009434


###### The author with the highest average book rating is Diana Gabaldon with a rating of 4.3.

##### Find the average number of text reviews among users who rated more than 50 books.

In [17]:
df=pd.read_sql_query(("""SELECT username, COUNT(rating_id) AS rating_number
                         FROM ratings
                         GROUP BY username
                         HAVING COUNT(rating_id) > 50 """), con=connection_string)
df

,username,rating_number
0,sfitzgerald,55
1,jennifermiller,53
2,xdavis,51
3,paul88,56
4,martinadam,56
5,richard89,55


In [18]:
df=pd.read_sql_query(("""SELECT AVG(review_count)  
                      FROM  (
                              SELECT username, COUNT(*) AS review_count
                              FROM reviews
                              WHERE username IN (
                                     SELECT username
                                     FROM ratings
                                     GROUP BY username
                                     HAVING COUNT(DISTINCT book_id) > 50
                                     )
                               GROUP BY username
                          )AS review_counts
                        """), con=connection_string)
df

,avg
0,24.333333


###### The average number of user text reviews with over 50 ratings is 24.

#### Conculsions
1. Most of the books were published in the 21st century.
2. The books with the highest number of reviews do not always have a high average rating. We also notice that the title of the book is repeated.
3. Penguin Books has published 42 books, which has the greatest number of books with more than          50 pages.
4. The author with the highest average book rating is Diana Gabaldon with a rating of 4.3.
5. The average number of user text reviews with over 50 ratings is 24.33333.